In [12]:
import json
import tiktoken
from collections import defaultdict
import numpy as np

In [3]:
def count_tokens_in_jsonl(file_path, model="gpt-4o-mini"):
    # Load the tokenizer for the specified model
    encoding = tiktoken.encoding_for_model(model)

    total_tokens = 0

    # Open JSONL-file and proces row by row
    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            data = json.loads(line)  # Load JSON-object
            messages = data.get("messages", [])
            
            for message in messages:
                content = message.get("content", "")
                total_tokens += len(encoding.encode(content))

    print(f"Total tokens in {file_path}: {total_tokens}")
    return total_tokens

In [4]:
file_train_path = 'multi_turn_conversation_train.jsonl'
file_val_path = 'multi_turn_conversation_val.jsonl'

In [5]:
count_tokens_in_jsonl(file_train_path)
count_tokens_in_jsonl(file_val_path)

Total tokens in multi_turn_conversation_train.jsonl: 1683
Total tokens in multi_turn_conversation_val.jsonl: 297


297

In [6]:
# Dataset validation
with open(file_train_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 19
First example:
{'role': 'system', 'content': 'You are a polite and professional assistant committed to delivering excellent service.'}
{'role': 'user', 'content': 'My internet is down and xfinity talkin about 2472 hours yall have the game messed up'}
{'role': 'assistant', 'content': 'I understand this is a frustrating experience please send a DM with your account information so I can look into this matter for you AD'}


In [9]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [10]:
def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [13]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "assistant" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    
print("Num examples missing assistant message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")

Num examples missing assistant message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 9
mean / median: 4.7368421052631575, 4.0
p5 / p95: 3.0, 8.2
